Import

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import cv2

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import adam_v2
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

**Initialise the training and validation generators:**


In [ ]:
train_dir = "/content/drive/MyDrive/ML Projects/Face/train"
val_dir = "/content/drive/MyDrive/ML Projects/Face/test"
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28719 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


**Build the convolution network architecture**

In [ ]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

**Compile and train the model:**

In [ ]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=adam_v2.Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
from keras.callbacks import ModelCheckpoint
model_path = "/content/drive/MyDrive/ML Projects/Face/face_predict.h5"
checkpoint = ModelCheckpoint(model_path,monitor = 'val_accuracy',verbose =1, save_best_only = True,mode ='max')
callbacks_list = [checkpoint]
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64,
        callbacks = callbacks_list)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if sys.path[0] == '':


Epoch 1/50
448/448 [==============================] - ETA: 0s - loss: 1.8018 - accuracy: 0.2591
Epoch 1: val_accuracy improved from -inf to 0.31892, saving model to /content/drive/MyDrive/ML Projects/Face/face_predict.h5
448/448 [==============================] - 74s 149ms/step - loss: 1.8018 - accuracy: 0.2591 - val_loss: 1.7182 - val_accuracy: 0.3189
Epoch 2/50
448/448 [==============================] - ETA: 0s - loss: 1.6260 - accuracy: 0.3687
Epoch 2: val_accuracy improved from 0.31892 to 0.41099, saving model to /content/drive/MyDrive/ML Projects/Face/face_predict.h5
448/448 [==============================] - 51s 115ms/step - loss: 1.6260 - accuracy: 0.3687 - val_loss: 1.5362 - val_accuracy: 0.4110
Epoch 3/50
448/448 [==============================] - ETA: 0s - loss: 1.5188 - accuracy: 0.4161
Epoch 3: val_accuracy improved from 0.41099 to 0.44545, saving model to /content/drive/MyDrive/ML Projects/Face/face_predict.h5
448/448 [==============================] - 51s 113ms/step - los

**Using openCV haarcascade xml detect the bounding boxes of faces in the webcam and predict the emotion:**

In [ ]:
cv2.ocl.setUseOpenCL(False)

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier('/home/shivam/.local/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2gray_frame)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    cv2.waitKey(1) & 0xFF == ord('q')
    cap.release()
    cv2.destroyAllWindows()
